In [1]:
import numpy as np
import pandas as pd
import MetaTrader5 as mt5

import statsmodels.api as sm
from pandas.tseries.offsets import BDay

import csv
import time
import math
from datetime import datetime, timedelta

import requests
import json

from ta.trend import ADXIndicator
import pandas_ta as ta

import vectorbt as vbt
from tqdm import tqdm

from multiprocessing import Process

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cycler

colors = cycler('color', ['669FEE', '66EE91', '9988DD', 'EECC55', '88BB44', 'FFBBBB'])
plt.rc('figure', facecolor='313233')
plt.rc('axes',  facecolor='313233', edgecolor='none', axisbelow=True, grid=True, prop_cycle=colors, labelcolor='gray')
plt.rc('grid', color='474A4A', linestyle='solid')
plt.rc('xtick', color='gray')
plt.rc('ytick', direction='out', color='gray')
plt.rc('legend', facecolor='313233', edgecolor='313233')
plt.rc('text')

In [3]:
# Twitter notification keys

chat_id = "6208180231"
api_key = "6274819941:AAF8U6FnGv5A0DcSBNXCpnPCJPciwSww8-A"

In [6]:
def send_telegram_message(message: str, chat_id: str, api_key: str,):
    responses = {}

    proxies = None
    headers = {'Content-Type': 'application/json',
                'Proxy-Authorization': 'Basic base64'}
    data_dict = {'chat_id': chat_id,
                 'text': message,
                 'parse_mode': 'HTML',
                 'disable_notification': True}
    data = json.dumps(data_dict)
    url = f'https://api.telegram.org/bot{api_key}/sendMessage'
    
    requests.packages.urllib3.disable_warnings()
    response = requests.post(url,
                             data=data,
                             headers=headers,
                             proxies=proxies,
                             verify=False)
    return response

In [7]:
def get_bars(symbol, start_date, end_date, timeframe=mt5.TIMEFRAME_M15):
    mt5.initialize()
    bars = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    df_bars = pd.DataFrame(bars)
    df_bars["time"] = pd.to_datetime(df_bars["time"], unit="s")
    return df_bars

In [8]:
def get_latest_bar(symbol, timeframe=mt5.TIMEFRAME_M15):
    mt5.initialize()
    bar = mt5.copy_rates_from(
        symbol, 
        timeframe, 
        datetime.now() + timedelta(hours = 3), 
        1)  # number of bars
    df_bar = pd.DataFrame(bar)
    df_bar["time"] = pd.to_datetime(df_bar["time"], unit="s")
    return df_bar

In [9]:
def get_prices(location):
    mt5.initialize()

    symbol_list = []
    for symbol in mt5.symbols_get():
        if location in symbol.path:
            symbol_list.append(symbol)
            
    return symbol_list

In [10]:
def new_bar_event(symbol, last_bar):
    mt5.initialize()
    
    # Fixing a formatting error when getting last_bar['time'].iloc[0]
    date_str = last_bar['time'].iloc[0]
    date_format = '%Y-%m-%d %H:%M:%S'

    last_bar_time_formatted = datetime.strptime(date_str, date_format)
    while True:
        new_bar = get_latest_bar(symbol)        
        if new_bar['time'].iloc[0] != last_bar_time_formatted:
            return new_bar
        
        time.sleep(300)

In [11]:
def get_num_of_lots(name, leverage=30, risk=0.01):
    ask_per_fraction = (mt5.symbols_get(name)[0].ask * mt5.symbols_get(name)[0].trade_contract_size) * mt5.symbols_get(name)[0].volume_min
    ask_per_fraction_m = ask_per_fraction
    capital_at_risk = mt5.account_info().balance * risk
    
    order_size = math.floor(capital_at_risk / ask_per_fraction_m) * mt5.symbols_get(symbol)[0].volume_min
    return  order_size

In [12]:
def find_filling_mode(symbol):
    mt5.initialize()
    
    for i in range(2):
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": mt5.symbol_info(symbol).volume_min,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info_tick(symbol).ask,
            "type_filling": i,
            "type_time": mt5.ORDER_TIME_GTC}
 
        result = mt5.order_check(request)
            
        if result.comment == "Done":
            break
 
    return i

In [13]:
def open_long(name, lot_size):
    mt5.initialize()
    filling_type = find_filling_mode(name)
    request = {
            "action" : mt5.TRADE_ACTION_DEAL,
            "symbol" : name,
            "volume" : lot_size,
            "type" : mt5.ORDER_TYPE_BUY,   # mt5.ORDER_TYPE_SELL
            "price" : mt5.symbol_info_tick(name).ask, # mt5.symbol_info_tick(symbol).bid 
            "deviation" : 10,
            "type_filling" : filling_type,
            "type_time " : mt5.ORDER_TIME_GTC
        }
    
    result = mt5.order_send(request)
    return result

In [14]:
def close_long(name, lot_size, order_id):
    mt5.initialize()
    filling_type = find_filling_mode(name)
    request = {
            "action" : mt5.TRADE_ACTION_DEAL,
            "symbol" : name,
            "position" : order_id,
            "volume" : lot_size,
            "type" : mt5.ORDER_TYPE_SELL,
            "price" : mt5.symbol_info_tick(name).bid, 
            "deviation" : 10,
            "type_filling" : filling_type,
            "type_time " : mt5.ORDER_TIME_GTC
        }
    
    result = mt5.order_send(request)
    return result

In [15]:
def open_short(name, lot_size):
    mt5.initialize()
    filling_type = find_filling_mode(name)
    request = {
            "action" : mt5.TRADE_ACTION_DEAL,
            "symbol" : name,
            "volume" : lot_size,
            "type" : mt5.ORDER_TYPE_SELL,
            "price" : mt5.symbol_info_tick(name).bid, 
            "deviation" : 10,
            "type_filling" : filling_type,
            "type_time " : mt5.ORDER_TIME_GTC
        }
    
    result = mt5.order_send(request)
    return result

In [16]:
def close_short(name, lot_size, order_id):
    mt5.initialize()
    filling_type = find_filling_mode(name)
    request = {
            "action" : mt5.TRADE_ACTION_DEAL,
            "symbol" : name,
            "position" : order_id,
            "volume" : lot_size,
            "type" : mt5.ORDER_TYPE_BUY,   # mt5.ORDER_TYPE_SELL
            "price" : mt5.symbol_info_tick(name).ask, # mt5.symbol_info_tick(symbol).bid 
            "deviation" : 10,
            "type_filling" : filling_type,
            "type_time " : mt5.ORDER_TIME_GTC
        }
        
    result = mt5.order_send(request)   # Sends order to broker
    return result  

In [17]:
def draw_graph(data, fast_ma, upper_threshold, lower_threshold):
    
    #fig, ax = plt.subplots() # fig : figure object, ax : Axes object
    plt.figure().set_figwidth(20)
    plt.plot(data)
    plt.plot(tp, color='r')
    plt.plot(upper_threshold, color='y')
    ax.set_xlabel('15 min ticks')
    ax.set_ylabel('price')
    ax.set_title("I am compatible with multithreading")
    plt.show()

    
    
    #plt.figure().set_figwidth(20)


In [18]:
def get_ticket(name):
    for position in mt5.positions_get():
        if position.symbol == name:
            return position.ticket
        
    return

In [19]:
def get_price_open(name):
    for position in mt5.positions_get():
        if position.symbol == name:
            return position.price_open
    
    return

In [20]:
def so_signal(k, d):
    if k > 80 and d > 80 and k < d:
        return -1
    elif k < 20 and d < 20 and k > d:
        return 1
    else:
        return 0

In [21]:
def backtest(symbol_data):
    symbol_close = symbol_data['close']
    
    try:
        filtered_symbol_close_cycle, filtered_symbol_close_trend = sm.tsa.filters.hpfilter(symbol_close)
    except ValueError as ve:
        print('Problem with filter')
        filtered_symbol_close_trend = symbol_close
        
      
    #window_sslow = 80
    window_slow = 40
    window_fast = 20
    slow_ma = filtered_symbol_close_trend.ewm(span=window_slow).mean()
    slow_std = filtered_symbol_close_trend.ewm(span=window_slow).std()
    fast_ma = filtered_symbol_close_trend.ewm(span=window_fast).mean()
    
    k_period = 14
    d_period = 3
    symbol_data['n_high'] = symbol_data['high'].rolling(k_period).max()
    symbol_data['n_low'] = symbol_data['low'].rolling(k_period).max()
    symbol_data['%k'] = (symbol_data['close'] - symbol_data['n_low']) * 100 / (symbol_data['n_high'] - symbol_data['n_low'])
    symbol_data['%D'] = symbol_data['%k'].rolling(d_period).mean()
    so = symbol_data.ta.stoch(high='high', low='low', k=14, d=3, append=True)        
    
    upper_threshold = slow_ma 
    lower_threshold = slow_ma
    
    
    entries = [False for i in range(len(symbol_close))]
    exits = [False for i in range(len(symbol_close))]
    
    LONG = 1
    SHORT = -1
    FREE = 0
    position_type = FREE
    position_entry = 0
    
    profit = 0
    for t in range(21, len(symbol_close)):
        
        # If Long
        if position_type == LONG:
            if (fast_ma.iloc[t] <= slow_ma.iloc[t]):
                position_type = FREE
                exits[t] = True
                profit += (symbol_close.iloc[t] - position_entry)
                
        elif position_type == SHORT:        
            if (fast_ma.iloc[t] >= slow_ma.iloc[t]):
                position_type = FREE 
                exits[t] = True
                profit += (position_entry - symbol_close.iloc[t])
                
        elif position_type == FREE:    
            # cross above -> long
            if (fast_ma.iloc[t] >= upper_threshold.iloc[t]) and so_signal(so['STOCHk_14_3_3'][t], so['STOCHd_14_3_3'][t]) == LONG:
                position_type = LONG
                position_entry = symbol_close.iloc[t]
                entries[t] = True
            
            # cross below -> short
            elif (fast_ma.iloc[t] <= lower_threshold.iloc[t]) and so_signal(so['STOCHk_14_3_3'][t], so['STOCHd_14_3_3'][t]) == SHORT:
                position_type = SHORT
                position_entry = symbol_close.iloc[t]
                entries[t] = True
    
    
    return entries, exits, profit

In [22]:
def trade_engine(name, lot_size, data, position_type):
    
    mt5.initialize()
    
    data_close = data['close']
    
    
    filtered_data_close_cycle, filtered_data_close_trend = sm.tsa.filters.hpfilter(data_close)
    
    
    # Construct slow MA, slow STD, and fast MA
    window_slow = 40
    window_fast = 20
    slow_ma = filtered_data_close_trend.ewm(span=window_slow).mean()
    slow_std = filtered_data_close_trend.ewm(span=window_slow).std()
    fast_ma = filtered_data_close_trend.ewm(span=window_fast).mean()
    
    
    # Contruct trade parameters
    upper_threshold = slow_ma 
    lower_threshold = slow_ma   
      
    
    k_period = 14
    d_period = 3
    data['n_high'] = data['high'].rolling(k_period).max()
    data['n_low'] = data['low'].rolling(k_period).max()
    data['%k'] = (data['close'] - data['n_low']) * 100 / (data['n_high'] - data['n_low'])
    data['%D'] = data['%k'].rolling(d_period).mean()
    so = data.ta.stoch(high='high', low='low', k=14, d=3, append=True)        
    
    LONG = 1
    FREE = 0
    SHORT = -1
    
    print(f'{namw}: {data.iloc[-1]}')
    draw_graph(data_close, fast_ma, upper_threshold, lower_threshold)
    time.sleep(1)

        
    # If Long
    if position_type == LONG:
        if (fast_ma.iloc[-1] <= slow_ma.iloc[-1]):
            ticket = get_ticket(name)
            result = close_long(name, lot_size, ticket)
            send_telegram_message(f'{name}: Closed Long Positions at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.', chat_id, api_key)
            print(f'{name}: Closed Long Positions at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.')
            position_type = FREE

    elif position_type == SHORT:        
        if (fast_ma.iloc[-1] >= slow_ma.iloc[-1]):
            # CLOSE SHORT POSITION AT A GAIN
            ticket = get_ticket(name)
            result = close_short(name, lot_size, ticket)
            send_telegram_message(f'{name}: Closed Short Positions at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.', chat_id, api_key)
            print(f'{name}: Closed Short Positions at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.')
            position_type = FREE

    elif position_type == FREE:    
        # cross above -> long
        if (fast_ma.iloc[-1] >= upper_threshold.iloc[-1]) and so_signal(so['STOCHk_14_3_3'].iloc[-1], so['STOCHd_14_3_3'].iloc[-1]) == LONG:
            # OPEN LONG POSITION
            result = open_long(name, lot_size)
            send_telegram_message(f'{name}: Openned Long Positions at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.', chat_id, api_key)
            print(f'{name}: Openned Long Positions at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.')
            position_type = LONG

        # cross below -> short
        elif (fast_ma.iloc[-1] <= lower_threshold.iloc[-1]) and so_signal(so['STOCHk_14_3_3'].iloc[-1], so['STOCHd_14_3_3'].iloc[-1]) == SHORT:
            result = open_short(name, lot_size)
            send_telegram_message(f'{name}: Openned Short Positions at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.', chat_id, api_key)
            print(f'{name}: Openned Short Positions at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.')
            position_type = SHORT

    return position_type
    

In [23]:
def momentum_trader(symbol_name):
    
    end_date = datetime.now() + timedelta(hours = 3)
    start_date = end_date - BDay(252)
    
    mt5.initialize()
    symbol_object = mt5.symbols_get(symbol_name)
    
    symbol_csv = symbol_name + '.csv'

    data = get_bars(
        symbol_name, 
        start_date, 
        end_date)

    data.to_csv(symbol_csv, index=False)

    data = pd.read_csv(symbol_csv)
    
    position_type = 0
    while True:
        last_bar = data.tail(1)
        new_bar = new_bar_event(symbol_name, last_bar)

        new_bar.to_csv(symbol_csv, mode='a', index=False, header=False)
        data = pd.read_csv(symbol_csv)

        lot_size = get_num_of_lots(symbol, risk=0.02)
        
        print(f'NEW BAR: {new_bar["time"].iloc[0]}')
        position_type = trade_engine(symbol_name, lot_size, data, position_type)

    

In [24]:
##################################################
# Set Parameters

end_date = datetime.now() + timedelta(hours = 3)
start_date = end_date - BDay(252)


mt5.initialize()

location = "Retail\\ETFs"

momentum_symbols = []

for symbol in tqdm(mt5.symbols_get()):
    if location in symbol.path:
        
        
        data = get_bars(symbol.name, start_date, end_date)
        time_series = data['close'] 

        entries, exits, profit = backtest(data)
        so_pf = vbt.Portfolio.from_signals(data['close'], entries=entries, exits=exits, freq='15m')
                                           
        if so_pf.sharpe_ratio() > 2:
            print(symbol.name)
            
            #print(so_pf.stats())
            momentum_symbols.append(symbol.name)                               
            print()
     

for symbol_name in momentum_symbols:
    trader = Process(target=momentum_trader, args=(symbol_name,))
    trader.start()
    time.sleep(2)

# MOST IMPORTANT PARAMETERS



##################################################

 80%|██████████████████████████████████████████████████████████████▎               | 968/1211 [00:04<00:01, 196.38it/s]

MSCI_Argentina_ETF_(AGT.N)



 85%|██████████████████████████████████████████████████████████████████▎           | 1030/1211 [00:15<00:27,  6.68it/s]

Lithium_ETF_(LIT.P)



 85%|██████████████████████████████████████████████████████████████████▌           | 1033/1211 [00:16<00:27,  6.48it/s]

...

 86%|███████████████████████████████████████████████████████████████████▎          | 1045/1211 [00:22<01:10,  2.34it/s]

iShares_Short_T-Bond_(SHV.O)



 87%|███████████████████████████████████████████████████████████████████▌          | 1049/1211 [00:22<00:36,  4.47it/s]

iShares_Semicond._(SOXX.O)



100%|██████████████████████████████████████████████████████████████████████████████| 1211/1211 [00:27<00:00, 44.24it/s]
